In [2]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats

In [3]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
with open("config_tests.yaml", "r") as stream:
    config_tests = yaml.safe_load(stream)

In [4]:
N_EPOCHS = 50
N_TRIALS = 10
BATCH_SIZE = 128

CONFIG = config_tests["assessment_proba_best"]

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [5]:
x, y = utils.load_data("../data/cifar-10-batches-py/data_batch_1")

In [6]:
# filepath_prefix = "../data/data2/seg_train/seg_train/"
#
# buildings = load_class_data(f"{filepath_prefix}buildings")
# street = load_class_data(f"{filepath_prefix}street")
# forest = load_class_data(f"{filepath_prefix}forest")
# glacier = load_class_data(f"{filepath_prefix}glacier")
# sea = load_class_data(f"{filepath_prefix}sea")
# mountain = load_class_data(f"{filepath_prefix}mountain")

In [7]:
# x = np.concatenate([buildings, street, forest, glacier, sea, mountain])
# y = np.concatenate(
#     [
#         [0] * buildings.shape[0],
#         [1] * street.shape[0],
#         [2] * forest.shape[0],
#         [3] * glacier.shape[0],
#         [4] * sea.shape[0],
#         [5] * mountain.shape[0],
#     ]
# )

In [8]:
x /= 255

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

n_classes = len(np.unique(y))
train_size = x_train.shape[0]

In [9]:
assessment_model = ClassifierModel(output_shape=n_classes, **models_hyperparameters["assessment_model"])

assessment_model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

assessment_model.fit(x_train, y_train, epochs=30, batch_size=BATCH_SIZE)

Epoch 1/30
63/63 [==============================] - 2s 16ms/step - loss: 2.4397 - accuracy: 0.1516
Epoch 2/30
63/63 [==============================] - 1s 16ms/step - loss: 2.1302 - accuracy: 0.2171
Epoch 3/30
63/63 [==============================] - 1s 16ms/step - loss: 1.9787 - accuracy: 0.2655
Epoch 4/30
63/63 [==============================] - 1s 16ms/step - loss: 1.8619 - accuracy: 0.3109
Epoch 5/30
63/63 [==============================] - 1s 16ms/step - loss: 1.7945 - accuracy: 0.3447
Epoch 6/30
63/63 [==============================] - 1s 17ms/step - loss: 1.7309 - accuracy: 0.3577
Epoch 7/30
63/63 [==============================] - 1s 16ms/step - loss: 1.6880 - accuracy: 0.3758
Epoch 8/30
63/63 [==============================] - 1s 16ms/step - loss: 1.6538 - accuracy: 0.3961
Epoch 9/30
63/63 [==============================] - 1s 17ms/step - loss: 1.6086 - accuracy: 0.3997
Epoch 10/30
63/63 [==============================] - 1s 21ms/step - loss: 1.5703 - accuracy: 0.4185
Epoch 11/

In [10]:
x_train_sorted = x_train[np.argsort(y_train)]
y_train_sorted = y_train[np.argsort(y_train)]
_, counts = np.unique(y_train_sorted, return_counts=True)

samples_proba = utils.calculate_proba(
    assessment_model, x_train_sorted, y_train_sorted, counts, CONFIG["negative_loss"]
)

In [ ]:
results = {}
# test_models = ["test_model_1", "test_model_2", "test_model_3"]
test_models = ["test_model_1"]


for test_model in test_models:
    print(test_model)
    model_scores = []

    for _ in tqdm(range(N_TRIALS)):
        model = ClassifierModel(output_shape=n_classes, **models_hyperparameters[test_model])
        model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

        for i in range(N_EPOCHS):
            n_samples = int(np.tanh(4 * (i + 1) / N_EPOCHS) * train_size)

            utils.chose_samples(n_samples, samples_proba, CONFIG["order_type"])
            samples_ids = np.random.choice(range(train_size), size=n_samples, replace=False)

            model.fit(
                x_train_sorted[samples_ids],
                y_train_sorted[samples_ids],
                validation_data=(x_val, y_val),
                epochs=1,
                batch_size=BATCH_SIZE,
                verbose=1,
            )
            
            if CONFIG["progressive"]:
                samples_proba = utils.calculate_proba(
                    model, x_train_sorted, y_train_sorted, counts, CONFIG["negative_loss"]
                )

        _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)

        model_scores.append(accuracy)

    results[test_model] = model_scores

test_model_1


  0%|          | 0/10 [00:00<?, ?it/s]

8/8 [==============================] - 0s 10ms/step - loss: 1.3287 - accuracy: 0.5070


 10%|█         | 1/10 [01:51<16:47, 111.94s/it]

8/8 [==============================] - 0s 10ms/step - loss: 1.8423 - accuracy: 0.3870


...

8/8 [==============================] - 0s 10ms/step - loss: 1.2546 - accuracy: 0.5230


 30%|███       | 3/10 [05:08<11:47, 101.03s/it]

8/8 [==============================] - 0s 9ms/step - loss: 1.3587 - accuracy: 0.4910


 40%|████      | 4/10 [06:44<09:55, 99.29s/it] 

21/47 [============>.................] - ETA: 0s - loss: 1.8540 - accuracy: 0.3021

In [56]:
results

{'test_model_1': [0.5180000066757202,
  0.5109999775886536,
  0.5189999938011169,
  0.414000004529953,
  0.5080000162124634]}

In [57]:
sum(results["test_model_1"]) / 5

0.49399999976158143

In [ ]:
rs1 = results["test_model_1"][:]

In [ ]:
rs2 = results["test_model_1"][:]

In [ ]:
stats.ttest_ind(rs1, rs2)